In [31]:
import nevis
import scipy
import scipy.optimize
import numpy as np
import matplotlib.pyplot as plt
import math
import pandas as pd
nevis.download_os_terrain_50()

Downloaded, unpacked, and processed file already found: Skipping.


# Randomly selecting points

Suppose we want the point to be within the circle centered at Ben Nevis with a radius R. If we randomly select points on the map in a uniform way, then the probability we hit this region is

$$
p = \frac {\pi R^2} {XY}.
$$

The probability of selecting N points, and none of them hitting the region is

$$
(1-p)^N.
$$

We want this probability to be less than $0.05$, and thus 

$$
N \ge \log_{1-p}(0.05) = \log_{1-\frac {\pi R^2} {XY}}(0.05).
$$

In [3]:
x_max, y_max = nevis.dimensions()

def random_select_n(R):
    return math.log(0.05, 1 - (R * R * math.pi) / (x_max * y_max))

for R in [1000, 100, 10]:
    print("Radius = {}, needed eval num = {:.1E}.".format(R, random_select_n(R)))

Radius = 1000, needed eval num = 8.7E+05.
Radius = 100, needed eval num = 8.7E+07.
Radius = 10, needed eval num = 8.7E+09.


In [26]:
f = nevis.linear_interpolant()

x_max, y_max = nevis.dimensions()
def random_search(n):
    print('number of evals: ', n)
    
    z = 0 
    sx, sy = 0, 0
    for _ in range(n):
        x = np.random.uniform(0, x_max)
        y = np.random.uniform(0, y_max)
        w = f(x, y)
        if w > z:
            z = w
            sx, sy = x, y

    return (sx, sy), z

In [30]:
df = []
for n in [1e4, 5e4, 1e5, 5e5]:
    (sx, sy), z = random_search(int(n))
    nevis.print_result(sx, sy, z)
    print()

    df.append({
        'evals': n,
        'height': z,
    })

number of evals:  10000
Good job!
You landed at an altitude of 1142m.
  https://opentopomap.org/#marker=15/57.096152/-3.672817
You are 775m from the nearest named hill top, "Cairn Lochan",
  ranked the 15th heighest in GB.

number of evals:  50000
Interesting!
You landed at an altitude of 1104m.
  https://opentopomap.org/#marker=15/57.085642/-3.687426
You are 1.4km from the nearest named hill top, "Sron na Lairige",
  ranked the 20th heighest in GB.

number of evals:  100000
Good job!
You landed at an altitude of 1109m.
  https://opentopomap.org/#marker=15/57.113722/-3.637046
You are 561m from the nearest named hill top, "Cairn Gorm",
  ranked the 9th heighest in GB.
  http://hillsummits.org.uk/htm_summit/525.htm

number of evals:  500000
Good job!
You landed at an altitude of 1257m.
  https://opentopomap.org/#marker=15/56.795476/-5.00065
You are 245m from the nearest named hill top, "Ben Nevis",
  ranked the 1st heighest in GB.
  http://hillsummits.org.uk/htm_summit/278.htm



In [32]:
pd.DataFrame(df)

,evals,height
0,10000.0,1142.094242
1,50000.0,1103.966841
2,100000.0,1108.897912
3,500000.0,1257.038215



## Selecting grid points

Now we use grid points. Suppose squares formed using neighbouring grid points has a side length $x$. Consider the minimum possible $x$ such that a grid square can cover a circle with a radius $R$. We get

$$
x = \sqrt 2 R.
$$

To make such grid points we need

$$
N \ge \frac {XY} {x^2} = \frac {XY} {2 R^2}.
$$



In [9]:
def grid_select_n(R):
    return x_max * y_max / (R * R * 2)

for R in [1000, 500, 100, 10]:
    print("Radius = {}, needed eval num = {:.1E}.".format(R, grid_select_n(R)))

Radius = 1000, needed eval num = 4.6E+05.
Radius = 500, needed eval num = 1.8E+06.
Radius = 100, needed eval num = 4.6E+07.
Radius = 10, needed eval num = 4.6E+09.


An experiment with grid searching:

In [34]:
f = nevis.linear_interpolant()
    

x_max, y_max = nevis.dimensions()
def grid_search(R):
    x_ = R * (2 ** 0.5)
    xs = np.linspace(0, x_max, math.ceil(x_max / x_))
    ys = np.linspace(0, y_max, math.ceil(y_max / x_))

    print('number of evals: ', len(xs) * len(ys))
    
    z = 0 
    sx, sy = 0, 0
    for x in xs:
        for y in ys:
            w = f(x, y)
            if w > z:
                z = w
                sx, sy = x, y

    return (sx, sy), z, len(xs) * len(ys)


In [36]:
df = []
for R in [5000, 1000, 500]:
    print(R)
    (sx, sy), z, n = grid_search(R)
    nevis.print_result(sx, sy, z)

    df.append({
        'radius': R,
        'evals': n,
        'height': z,
    })

5000
number of evals:  18216
Good job!
You landed at an altitude of 1087m.
  https://opentopomap.org/#marker=15/57.107529/-3.41673
You are 587m from the nearest named hill top, "Mullach Lochan nan Gabhar",
  ranked the 60th heighest in GB.
1000
number of evals:  455400
Good job!
You landed at an altitude of 1276m.
  https://opentopomap.org/#marker=15/57.073048/-3.667768
You are 308m from the nearest named hill top, "Ben Macdui",
  ranked the 2d heighest in GB.
  http://hillsummits.org.uk/htm_summit/518.htm
500
number of evals:  1820610
Good job!
You landed at an altitude of 1277m.
  https://opentopomap.org/#marker=15/57.078537/-3.731386
You are 184m from the nearest named hill top, "Braeriach",
  ranked the 3d heighest in GB.


In [37]:
pd.DataFrame(df)

,radius,evals,height
0,5000,18216,1087.115622
1,1000,455400,1275.820270
2,500,1820610,1277.196058


The number of evaluations grow too fast to actually try $R < 500$.

## Nelder-Mead with grid starting points

In [19]:
points = []

def wrapper(u):
    x, y = u
    points.append(u)
    return -f(x, y)

def nelder_mead_grid(s): 
    global points

    # s is the side length of the grid
    xs = np.linspace(0, x_max, math.ceil(x_max / s))
    ys = np.linspace(0, y_max, math.ceil(y_max / s))

    print('number of iterations: ', len(xs) * len(ys))

    points = []

    x_res, y_res, z_res = 0, 0, 0

    for x in xs:
        for y in ys:
            result = scipy.optimize.minimize(
                wrapper, 
                (x, y), 
                method='Nelder-Mead',
                bounds=((0, x_max), (0, y_max))
            )

            if -result.fun > z_res:
                x_res, y_res = result.x
                z_res = -result.fun
    
    return x_res, y_res, z_res, points
            
            

In [38]:
df = []
for s in [1e6, 5e5, 1e5, 5e4, 3e4]:
    print(s)
    x, y, z, points = nelder_mead_grid(int(s))
    nevis.print_result(x, y, z)
    print('number of evals:', len(points))
    print()

    df.append({
        'side length': s,
        'evals': len(points),
        'height': z,
    })

1000000.0
number of iterations:  2
Good job!
You landed at an altitude of 31m.
  https://opentopomap.org/#marker=15/49.966447/-6.348534
You are 319m from the nearest named hill top, "King Charles's Castle (Tresco)",
  ranked the 20663d heighest in GB.
  http://hillsummits.org.uk/htm_summit/17889.htm
number of evals: 331

500000.0
number of iterations:  6
Good job!
You landed at an altitude of 372m.
  https://opentopomap.org/#marker=15/55.573014/-5.569342
You are 660m from the nearest named hill top, "Beinn an Tuirc",
  ranked the 7324th heighest in GB.
  http://hillsummits.org.uk/htm_summit/1410.htm
number of evals: 1013

100000.0
number of iterations:  91
Congratulations!
You landed at an altitude of 1294m.
  https://opentopomap.org/#marker=15/57.078398/-3.728443
You are 10m from the nearest named hill top, "Braeriach",
  ranked the 3d heighest in GB.
number of evals: 16644

50000.0
number of iterations:  364
Congratulations!
You landed at an altitude of 1294m.
  https://opentopomap.o

In [39]:
pd.DataFrame(df)

,side length,evals,height
0,1000000.0,331,30.782219
1,500000.0,1013,371.601916
2,100000.0,16644,1293.900015
3,50000.0,65274,1293.900023
4,30000.0,194954,1309.099975
